In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3507923569139430756, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 18052572022818578325
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time
import math
import os
import pandas as pd
from keras.callbacks import EarlyStopping
import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize
from keras.utils.np_utils import to_categorical
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [5]:
batch_size = 32
img_height, img_width = 128, 128
input_shape = (img_height, img_width, 3)
epochs = 15

In [6]:
train_dir = 'data/train/'
test_dir = 'data/test'

In [7]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split= 0.3,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 34909 images belonging to 6 classes.
Found 14958 images belonging to 6 classes.
Found 13578 images belonging to 6 classes.


In [8]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 34909
nb_validation_samples: 14958
nb_test_samples: 13578

predict_size_train: 1091
predict_size_validation: 468
predict_size_test: 425

 num_classes: 6


In [9]:
# vgg19_weights ="../input/full-keras-pretrained-no-top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_weights ="../input/full-keras-pretrained-no-top//inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
# vgg16_weights ="../input/full-keras-pretrained-no-top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet201_weights ="../input/full-keras-pretrained-no-top/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet121_weights ="../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
# resenet50_weights ="../input/full-keras-pretrained-no-top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_resnet_v2_weights ="../input/full-keras-pretrained-no-top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
# nasnet_weights ="../input/full-keras-pretrained-no-top/nasnet_large_no_top.h5"
# nasnet_mobile_weights ="../input/full-keras-pretrained-no-top/nasnet_mobile_no_top.h5"
# mobilenet_weights ="../input/full-keras-pretrained-no-top/mobilenet_1_0_224_tf_no_top.h5"

In [10]:
# from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
# from keras.applications.vgg16 import VGG16
# from keras.applications.xception import Xception, preprocess_input
# from keras.applications import DenseNet201
# from keras.applications import DenseNet121
# from keras.applications import ResNet50
# from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from keras.applications import NASNetLarge, NASNetMobile
# from keras.applications import MobileNet

bottleneck_final_model=InceptionV3(weights="imagenet", include_top=False, pooling = "avg")

In [11]:
# os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "InceptionV3_descriptors"

In [12]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train, max_q_size=1, pickle_safe=False)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [13]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [14]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [15]:

import tensorflow as tf
from keras.backend.tensorflow_backend import get_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import set_session

def reset_keras_tf_session():
    """
    this function clears the gpu memory and set the 
    tf session to not use the whole gpu
    """
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

#     config = tf.ConfigProto()
#     config.gpu_options.allow_growth = True
#     set_session(tf.Session(config=config))

reset_keras_tf_session()

In [16]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [17]:
space = {
         'lr': hp.choice('lr',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.9]),
         'beta_2':hp.choice('beta_2',[0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.999,0.99,0.995]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [18]:
from sklearn.metrics import accuracy_score
def f_nn(params):   
    print ('Parameters testing: ', params)
  
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0)
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = Sequential()
    # model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.7))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(validation_data, validation_labels),
                        verbose= 2,
                        callbacks=get_callbacks(params))

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]
#     cm = confusion_matrix(y_pred=predictions, y_true=y_true)

    print('Test Accuracy: {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [19]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [20]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals = 300, trials=trials)
print(best)

Parameters testing:  {'beta_1': 0.7, 'beta_2': 0.4, 'lr': 0.01}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 12.3868 - acc: 0.2296 - val_loss: 12.4695 - val_acc: 0.2264
Epoch 2/15
 - 4s - loss: 12.4604 - acc: 0.2264 - val_loss: 12.3919 - val_acc: 0.2312
Epoch 3/15
 - 4s - loss: 12.3457 - acc: 0.2331 - val_loss: 12.3835 - val_acc: 0.2316
Epoch 4/15
 - 4s - loss: 12.3280 - acc: 0.2340 - val_loss: 13.5928 - val_acc: 0.1564
Epoch 5/15
 - 4s - loss: 12.2973 - acc: 0.2356 - val_loss: 12.4695 - val_acc: 0.2264
Epoch 6/15
 - 4s - loss: 12.3412 - acc: 0.2327 - val_loss: 12.4695 - val_acc: 0.2264
Epoch 00006: early stopping
14958/14958 [==============================] - 1s 36us/step
Validation Accuracy: 22.6367%
Validation Loss: 12.469487922557265
Test Accuracy: 0.2267638827515098
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.9, 

 - 4s - loss: 0.6911 - acc: 0.8100 - val_loss: 1.1374 - val_acc: 0.6873
Epoch 5/15
 - 4s - loss: 0.7417 - acc: 0.8095 - val_loss: 1.1226 - val_acc: 0.6976
Epoch 6/15
 - 4s - loss: 0.7385 - acc: 0.8153 - val_loss: 1.5724 - val_acc: 0.6949
Epoch 7/15
 - 4s - loss: 0.7541 - acc: 0.8174 - val_loss: 1.5917 - val_acc: 0.6464
Epoch 8/15
 - 4s - loss: 0.7576 - acc: 0.8200 - val_loss: 1.5260 - val_acc: 0.6816
Epoch 00008: early stopping
14958/14958 [==============================] - 1s 40us/step
Validation Accuracy: 68.1575%
Validation Loss: 1.5259628993894887
Test Accuracy: 0.8025482397996759
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.1, 'beta_2': 0.99, 'lr': 0.0001}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 0.7744 - acc: 0.7268 - val_loss: 0.8680 - val_acc: 0.6703
Epoch 2/15
 - 4s - loss: 0.5269 - acc: 0.8134 - val_loss:

Parameters testing:  {'beta_1': 0.2, 'beta_2': 0.99, 'lr': 0.1}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 12.6084 - acc: 0.2171 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 2/15
 - 4s - loss: 12.6238 - acc: 0.2168 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 3/15
 - 4s - loss: 12.6238 - acc: 0.2168 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 4/15
 - 4s - loss: 12.6238 - acc: 0.2168 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 00004: early stopping
14958/14958 [==============================] - 1s 37us/step
Validation Accuracy: 21.6807%
Validation Loss: 12.623578551938024
Test Accuracy: 0.2154956547356017
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.2, 'beta_2': 0.3, 'lr': 1e-07}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 3.2675 - acc: 0.1963 - val_loss: 2.1786 - val_acc: 0.2374
Epoch 2/1

 - 4s - loss: 2.6459 - acc: 0.2183 - val_loss: 1.7690 - val_acc: 0.2469
Epoch 6/15
 - 4s - loss: 2.5305 - acc: 0.2346 - val_loss: 1.7161 - val_acc: 0.2642
Epoch 7/15
 - 4s - loss: 2.4196 - acc: 0.2504 - val_loss: 1.6748 - val_acc: 0.2829
Epoch 8/15
 - 4s - loss: 2.3476 - acc: 0.2574 - val_loss: 1.6415 - val_acc: 0.2976
Epoch 9/15
 - 4s - loss: 2.2623 - acc: 0.2716 - val_loss: 1.6142 - val_acc: 0.3105
Epoch 10/15
 - 4s - loss: 2.2008 - acc: 0.2818 - val_loss: 1.5908 - val_acc: 0.3208
Epoch 11/15
 - 4s - loss: 2.1383 - acc: 0.2940 - val_loss: 1.5706 - val_acc: 0.3303
Epoch 12/15
 - 4s - loss: 2.0932 - acc: 0.3014 - val_loss: 1.5525 - val_acc: 0.3396
Epoch 13/15
 - 4s - loss: 2.0266 - acc: 0.3120 - val_loss: 1.5365 - val_acc: 0.3474
Epoch 14/15
 - 4s - loss: 1.9792 - acc: 0.3243 - val_loss: 1.5217 - val_acc: 0.3550
Epoch 15/15
 - 4s - loss: 1.9374 - acc: 0.3363 - val_loss: 1.5090 - val_acc: 0.3610
14958/14958 [==============================] - 1s 39us/step
Validation Accuracy: 36.1011%
Va

Epoch 13/15
 - 4s - loss: 0.3173 - acc: 0.8858 - val_loss: 0.8975 - val_acc: 0.7138
Epoch 14/15
 - 4s - loss: 0.3131 - acc: 0.8868 - val_loss: 0.8757 - val_acc: 0.7184
Epoch 15/15
 - 4s - loss: 0.3034 - acc: 0.8911 - val_loss: 0.8761 - val_acc: 0.7121
Epoch 00015: early stopping
14958/14958 [==============================] - 1s 39us/step
Validation Accuracy: 71.2060%
Validation Loss: 0.8761261458062124
Test Accuracy: 0.843791427308882
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.8, 'beta_2': 0.995, 'lr': 0.0001}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 6s - loss: 0.7818 - acc: 0.7227 - val_loss: 0.8972 - val_acc: 0.6523
Epoch 2/15
 - 4s - loss: 0.5260 - acc: 0.8119 - val_loss: 0.8300 - val_acc: 0.6888
Epoch 3/15
 - 4s - loss: 0.4721 - acc: 0.8301 - val_loss: 0.8700 - val_acc: 0.6847
Epoch 4/15
 - 4s - loss: 0.4385 - acc: 0.84

...

Epoch 3/15
 - 10s - loss: 12.6234 - acc: 0.2168 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 4/15
 - 11s - loss: 12.6243 - acc: 0.2168 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 00004: early stopping
14958/14958 [==============================] - 2s 133us/step
Validation Accuracy: 21.6807%
Validation Loss: 12.623578551938024
Test Accuracy: 0.2154956547356017
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.9, 'beta_2': 0.3, 'lr': 0.01}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 22s - loss: 12.5097 - acc: 0.2230 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 2/15
 - 10s - loss: 12.5199 - acc: 0.2232 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 3/15
 - 11s - loss: 12.4895 - acc: 0.2251 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 4/15
 - 10s - loss: 12.5139 - acc: 0.2236 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 00004: early stopping
14

Epoch 6/15
 - 11s - loss: 1.1532 - acc: 0.5772 - val_loss: 1.2183 - val_acc: 0.5024
Epoch 7/15
 - 10s - loss: 1.0838 - acc: 0.6074 - val_loss: 1.1847 - val_acc: 0.5162
Epoch 8/15
 - 10s - loss: 1.0327 - acc: 0.6291 - val_loss: 1.1551 - val_acc: 0.5277
Epoch 9/15
 - 11s - loss: 0.9960 - acc: 0.6433 - val_loss: 1.1348 - val_acc: 0.5384
Epoch 10/15
 - 10s - loss: 0.9607 - acc: 0.6538 - val_loss: 1.1107 - val_acc: 0.5491
Epoch 11/15
 - 11s - loss: 0.9259 - acc: 0.6687 - val_loss: 1.0901 - val_acc: 0.5599
Epoch 12/15
 - 10s - loss: 0.8935 - acc: 0.6819 - val_loss: 1.0761 - val_acc: 0.5660
Epoch 13/15
 - 10s - loss: 0.8787 - acc: 0.6874 - val_loss: 1.0637 - val_acc: 0.5720
Epoch 14/15
 - 10s - loss: 0.8582 - acc: 0.6932 - val_loss: 1.0534 - val_acc: 0.5786
Epoch 15/15
 - 10s - loss: 0.8419 - acc: 0.7005 - val_loss: 1.0381 - val_acc: 0.5846
14958/14958 [==============================] - 2s 140us/step
Validation Accuracy: 58.4637%
Validation Loss: 1.038098996384352
Test Accuracy: 0.70886728531

Epoch 15/15
 - 11s - loss: 0.4630 - acc: 0.8373 - val_loss: 0.8612 - val_acc: 0.6874
14958/14958 [==============================] - 2s 136us/step
Validation Accuracy: 68.7391%
Validation Loss: 0.8611827511058954
Test Accuracy: 0.8093239063190455
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.9, 'beta_2': 0.8, 'lr': 0.01}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 23s - loss: 11.5925 - acc: 0.2785 - val_loss: 12.2574 - val_acc: 0.2391
Epoch 2/15
 - 11s - loss: 12.0786 - acc: 0.2500 - val_loss: 12.1164 - val_acc: 0.2482
Epoch 3/15
 - 11s - loss: 11.9961 - acc: 0.2552 - val_loss: 12.2006 - val_acc: 0.2428
Epoch 4/15
 - 11s - loss: 12.0069 - acc: 0.2537 - val_loss: 12.1501 - val_acc: 0.2461
Epoch 5/15
 - 10s - loss: 12.0217 - acc: 0.2479 - val_loss: 12.6109 - val_acc: 0.2175
Epoch 00005: early stopping
14958/14958 [==================

Epoch 7/15
 - 11s - loss: 0.5835 - acc: 0.7908 - val_loss: 0.8873 - val_acc: 0.6647
Epoch 8/15
 - 11s - loss: 0.5563 - acc: 0.8003 - val_loss: 0.8664 - val_acc: 0.6754
Epoch 9/15
 - 10s - loss: 0.5399 - acc: 0.8083 - val_loss: 0.8687 - val_acc: 0.6772
Epoch 10/15
 - 10s - loss: 0.5236 - acc: 0.8152 - val_loss: 0.8587 - val_acc: 0.6777
Epoch 11/15
 - 10s - loss: 0.5096 - acc: 0.8187 - val_loss: 0.8562 - val_acc: 0.6826
Epoch 12/15
 - 10s - loss: 0.5014 - acc: 0.8208 - val_loss: 0.8581 - val_acc: 0.6814
Epoch 13/15
 - 10s - loss: 0.4907 - acc: 0.8273 - val_loss: 0.8505 - val_acc: 0.6837
Epoch 14/15
 - 10s - loss: 0.4790 - acc: 0.8310 - val_loss: 0.8580 - val_acc: 0.6838
Epoch 15/15
 - 11s - loss: 0.4729 - acc: 0.8323 - val_loss: 0.8496 - val_acc: 0.6877
14958/14958 [==============================] - 2s 149us/step
Validation Accuracy: 68.7726%
Validation Loss: 0.8495881895435448
Test Accuracy: 0.8063043158049786
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_

Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 24s - loss: 12.4140 - acc: 0.2292 - val_loss: 12.3919 - val_acc: 0.2312
Epoch 2/15
 - 10s - loss: 12.3800 - acc: 0.2319 - val_loss: 12.3919 - val_acc: 0.2312
Epoch 3/15
 - 11s - loss: 12.3916 - acc: 0.2312 - val_loss: 12.3919 - val_acc: 0.2312
Epoch 4/15
 - 11s - loss: 12.3920 - acc: 0.2312 - val_loss: 12.3919 - val_acc: 0.2312
Epoch 00004: early stopping
14958/14958 [==============================] - 2s 164us/step
Validation Accuracy: 23.1181%
Validation Loss: 12.391903829807298
Test Accuracy: 0.23037266165856532
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.3, 'beta_2': 0.9, 'lr': 0.0001}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 22s - loss: 0.7718 - acc: 0.7275 - val_loss: 0.9074 - val_acc: 0.6623
Epoch 2/15
 - 10s - loss: 0.5338 - acc: 0.8129 - val_loss: 0.8652